In [ ]:
import logging, os
import nest_asyncio
from telegram import Update, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters, CallbackQueryHandler
import openpyxl
# Assuming you have a 'keys.py' file with your token
import my_keys

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

def load_workbook_and_sheet():
    global wb, sheet
    wb = openpyxl.load_workbook(data_base_filename)
    sheet = wb.active

data_base_filename = 'db.xlsx'

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Load your Excel workbook
load_workbook_and_sheet()

check_admin = False
    
# Function to check if the user is the bot administrator
def is_admin(update: Update, context: ContextTypes.DEFAULT_TYPE) -> bool:
    user = update.effective_user
    return user.id == my_keys.admin_id

async def upload_excel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if check_admin and not is_admin(update, context):
        await update.message.reply_text("You're not authorized to use this command.")
        return
    
    await update.message.reply_text("Please enter the password:")
    context.user_data['waiting_for_password'] = True
    return

async def cancel_upload(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data['waiting_for_file'] = False
    context.user_data['waiting_for_password'] = False
    await update.message.reply_text("The uploading canceled")
    
async def downloader(update, context):
    if context.user_data['waiting_for_password']:
        await update.message.reply_text("Before uploading please enter the password (Use /cancel_upload to cancel the upload if you want to): ")
        return
    
    if not context.user_data['waiting_for_file']:
        await update.message.reply_text("Please use /upload_excel command to upload files")
        return
    
    # Download file
    fileName = update.message.document.file_name
    new_file = await update.message.effective_attachment.get_file()
    await new_file.download_to_drive(fileName)
    os.replace(fileName, data_base_filename)
    
    context.user_data['waiting_for_file'] = False
    
    # Update the file
    load_workbook_and_sheet()
    
    # Acknowledge file received
    await update.message.reply_text(f"{fileName} saved successfully")
    return

# Function to read column names from the second column onwards
def get_column_names():
    return [cell.value for cell in sheet[2][1:]]

# Command handler for /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Send me a word, and I\'ll find its value in the Excel file.')
    context.user_data['waiting_for_password'] = False
    context.user_data['waiting_for_file'] = False
    
# Message handler for word input
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if context.user_data.get('waiting_for_password'):
        try:
            if update.message.text == str(my_keys.password):
                context.user_data['waiting_for_file'] = True
                context.user_data['waiting_for_password'] = False
                await update.message.reply_text("Password accepted. Please upload the Excel file.")
            else:
                await update.message.reply_text("Incorrect password. Access denied.")
                context.user_data['waiting_for_password'] = False
        except Exception as e:
            logging.error(f"Error in handle_password: {e}")
            context.user_data['waiting_for_password'] = False
        return
    
    if context.user_data['waiting_for_file']:
        await update.message.reply_text("Please upload the Excel file. Use /cancel_upload to cancel the upload")
        return
    
    
    word = update.message.text
    # Search for the word in the Excel file
    for row in sheet.iter_rows(min_row=2, values_only=True):
        if str(row[0]) == word:
            # Create an inline keyboard with column names
            keyboard = [
                [InlineKeyboardButton(column_name, callback_data=f"{word}:{column_name}")]
                for column_name in get_column_names()[:3]
            ]
            reply_markup = InlineKeyboardMarkup(keyboard)
            await update.message.reply_text('Select a column:', reply_markup=reply_markup)
            return
    await update.message.reply_text('Word not found in the Excel file.')

# Callback query handler for column selection
async def handle_button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    query.answer()
    word, column_name = query.data.split(':')
    # Find the row with the word and get the value from the selected column
    for row in sheet.iter_rows(min_row=2, values_only=True):
        if str(row[0]) == word:
            column_index = get_column_names().index(column_name) + 1 # +1 because columns start from 1
            value = row[column_index]
            await query.edit_message_text(f'The value for "{word}" in column "{column_name}" is: {value}')
            return

if __name__ == '__main__':
    application = ApplicationBuilder().token(my_keys.token).build()
    
    # Command handler for /start
    start_handler = CommandHandler('start', start)
    application.add_handler(start_handler)
    
    # Message handler for word input
    message_handler = MessageHandler(filters.TEXT & ~filters.COMMAND & ~filters.ATTACHMENT, handle_message)
    application.add_handler(message_handler)
    
    upload_excel_handler = CommandHandler('upload_excel', upload_excel)
    application.add_handler(upload_excel_handler)
    
    upload_excel_handler = CommandHandler('cancel_upload', cancel_upload)
    application.add_handler(upload_excel_handler)
    
    application.add_handler(MessageHandler(filters.ATTACHMENT, downloader))
    
    # Callback query handler for column selection
    button_handler = CallbackQueryHandler(handle_button)
    application.add_handler(button_handler)
    
    # Run the bot
    application.run_polling()